# Main

> Module die alle functies die nodig zijn om de snelste route te vinden langs alle peilbuizen gegroepeerd per project.

In de huidige code gebruiken we de Openrouteservice API die publiekelijk gehost wordt. Deze heeft een aantal restricties. Zie https://openrouteservice.org/restrictions/.
De enige restrictie waar we in deze code last van hebben is het maximaal aantal locaties dat meegenomen kan worden in de optimalisatie. Volgens de documentatie is dat 50. Maar uit mijn testen blijkt het 70 te zijn.

Verder gebruiken we de Python client (https://openrouteservice-py.readthedocs.io/en/latest/) en construeren we dus niet zelf de curl calls.

In [ ]:
#| default_exp main

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import yaml
import pandas as pd
import logging
from datetime import datetime
from fastcore.utils import Path
from tqdm import tqdm

from project.data_get import get_data_from_azuresql, load_pickle
from project.route_get import create_optimized_route
from project.data_export import get_waypoint_coords, convert_routejson_to_df, create_route_prev_next, google_maps_route_url
from project.utils import get_project_root, make_filesystem_friendly, save_route_url, setup_logging

## Prepare the script with exceptions and settings

Define the custom exceptions

In [ ]:
#| export
class RouteCreationError(Exception):
    """Exception raised when a route cannot be created."""
    pass

Get the settings from the settings.yaml file.

In [ ]:
#| export
project_root = get_project_root()

with open(project_root / 'settings.yaml', 'r') as f:
    settings = yaml.safe_load(f)

Get the data from the Azure SQL database in a pandas DataFrame or from a previously stored pickle file.

In [ ]:
#| export
use_pickle=settings['files']['use_pickle_input']

if use_pickle:
    peilbuizen_df = load_pickle(settings['files']['pickle_file_input'])
else:
    peilbuizen_df = get_data_from_azuresql(sql_statement=settings['sql_statement']['peilbuizen'], 
                                       save_to_pickle=settings['files']['save_to_pickle'])

df_grouped = peilbuizen_df.groupby('project')

In [ ]:
df_grouped.groups.keys()

dict_keys(['---', 'Bagger Dedemsvaart', 'Baggeren Beilervaart', 'Beekmaatregelen Reest', 'Boetelerveld_nw', 'Brongeb VL-AA', 'Dijken', 'Droogte Onderzoek NL fase 2 - bodemvochtmetingen', 'Droogtemeetnet WDOD', 'Ecologisch Effect Beekherstel Middenloop Vledder Aa Fase 1', 'Holtingerveld _Ootmaanlanden en Koningschut', 'Inrichting Dwingelderveld', 'Koekoekspolder grondwatermeetnet', 'Mastenbroek-IJssel', 'Meetnet De Wijk II', 'Nieuwveense landen', 'Nijstad Hoogeveen', 'Olde Maten en Veerslootlanden', 'Oldematen Reevediep', 'Oude Diep', 'Oude Diep Mantinge Bos_Zand', 'Oude Diep_Roode Brand', 'Oude Willem', 'Overijsselskanaal Deventer-Raalte', 'Paddenpol Zwolle-Olst', 'Primair meetnet blok 1', 'Primair meetnet blok 2', 'Primair meetnet blok 3', 'Randzone Ossenzijl Steenwijk', 'Reevediep', 'Reparatie meetpunten', 'Steenwijk_Kallenkote', 'Varsenerveld', 'Vecht', 'Vechterweerd', 'Vledder en Leierhooilanden', 'Wabos-KRW', 'ZUIDWOLDE-ZUID', 'Zandwetering Olst-Zuid_waterberging', 'dijken Stadsdi

In [ ]:
# Test one project group
test_df_group = df_grouped.get_group('Boetelerveld_nw')
test_name_group = 'Boetelerveld_nw'

# Test random sample
df_random_test = peilbuizen_df.sample(n=35)

In [ ]:
test_df_group.head()

,Id,Longitude,latitude,latest_measure_date,project
30,27HG800A;412187,6.314479,52.369582,2024-03-28 09:58:57,Boetelerveld_nw
31,27HG901A;412271,6.335889,52.366841,2024-03-28 10:27:15,Boetelerveld_nw
98,28CG001A;414791,6.342413,52.363634,2024-03-28 11:46:21,Boetelerveld_nw
243,27HG70G;411935,6.339147,52.362645,2024-03-28 11:11:21,Boetelerveld_nw
244,27HG902C;412313,6.340380,52.353098,2024-03-29 07:22:01,Boetelerveld_nw


## Start calculating the routes and storing the results

For every project in the peilbuizen dataframe, the shortest route is calculated and saved to an Excel file and an URL file.

In [ ]:
#| export

def create_group_route(start_address: str, # De startlocatie en eindlocatie opgegeven als adres
                       group_df: pd.DataFrame, # De pandas dataframe met peilbuizen in de groep, bijv. op basis van project
                       route_profile: str, # Het route profiel (transport methode) waarvoor de route berekend moet worden
                       project_name: str, # Naam van de groep, bijv. Project Vecht
                       output_dir: str = 'output', # Locatie waar de resultaten en log-file opgeslagen worden. Als niks wordt opgegeven komen de resultaten in de output folder.
                       current_date: str = None, # De datum. Als niks wordt opgegeven, wordt de huidige datum gebruikt.
                       ) -> None:
    """
    Create optimized route for a group and save results
    """
    if current_date is None:
        current_date = datetime.now().strftime('%Y-%m-%d')
        
    # Create output directory
    output_path = Path(output_dir)
    output_path.mkdir(parents=True, exist_ok=True)
    
    try:
        # Get optimized route
        route_json = create_optimized_route(start_address, group_df, route_profile)
        route_coords = get_waypoint_coords(route_json)
        
        # Create route dataframe
        optimized_route = convert_routejson_to_df(route_coords, group_df)
        route_table = create_route_prev_next(optimized_route)
        
        # Create filenames
        safe_project_name = make_filesystem_friendly(project_name)
        base_filename = f"peilbuizenroute_{safe_project_name}_{current_date}"
        
        # Save Excel file
        excel_path = output_path / f"{base_filename}.xlsx"
        route_table.to_excel(excel_path)
        
        # Save URL shortcut
        url = google_maps_route_url(route_coords)
        save_route_url(url, output_dir, f"{base_filename}.url")
        
    except Exception as e:
        raise RouteCreationError(f"Error processing group {project_name}: {str(e)}") from e


In [ ]:
create_group_route(start_address=settings['calculation']['startlocation'],
                    group_df=test_df_group,
                    route_profile=settings['calculation']['distance_calculation_method'],
                    project_name=test_name_group,
                    output_dir=Path(settings['files']['path_results']) / 'test')


In [ ]:
#| export

def process_peilbuizen_routes(df: pd.DataFrame, # Dataframe van alle peilbuis groepen waarvoor route berekend wordt
                            start_address: str, # De startlocatie en eindlocatie opgegeven als adres
                            route_profile: str, # De route profiel (transport methode) waarvoor de route berekend moet worden
                            output_dir: str = 'output' # Locatie waar de resultaten en log-file opgeslagen worden. Als niks wordt opgegeven komen de resultaten in de output folder.
                            ) -> None:
    """
    Process peilbuizen dataframe, create optimized routes for each group,
    and save results to Excel and URL files
    
    Args:
        df: DataFrame with peilbuizen data
        start_address: Starting location address
        route_profile: De route profiel (transport methode) waarvoor de route berekend moet worden
        output_dir: Directory to save output files
    """
    # Create output directory and setup logging
    output_path = Path(output_dir)
    output_path.mkdir(parents=True, exist_ok=True)
    current_date = datetime.now().strftime('%Y-%m-%d')
    logger = setup_logging(output_dir, current_date)
        
    # Process each group
    grouped = df.groupby('project')
    total_groups = len(grouped)
    
    for project_name, group_df in tqdm(grouped, total=total_groups,desc="Processing peilbuis routes per project."):
        if len(group_df) < 4:
            msg = f"Project {project_name} heeft 3 of minder locaties. Optimale route uitrekenen is zinloos."
            print(msg)
            logger.info(msg)
        elif len(group_df) > 70:
            msg = f"Project {project_name} heeft meer dan 70 locaties. Optimale route uitrekenen is niet mogelijk op de publieke Openrouteservice. Oplossing: verdeel het project in kleinere delen, zet de Openrouteservice op eigen omgeving."
            print(msg)
            logger.info(msg)
        else:
            #TODO: Als deze functie een error returned, dus geen route vindt, komt dat niet in logfiles.
            try:
                create_group_route(
                    start_address=start_address,
                    group_df=group_df,
                    route_profile=route_profile,
                    project_name=project_name,
                    output_dir=output_dir,
                    current_date=current_date
                )
                logger.info(f"Successfully processed route for project: {project_name}")
            except RouteCreationError as e:
                error_msg = str(e)
                print(error_msg)
                logger.error(error_msg)
            except Exception as e:
                error_msg = f"Error processing project {project_name}: {str(e)}"
                print(error_msg)
                logger.error(error_msg)

In [ ]:
# Test run voor enkele project van een deel van alle peilbuizen
process_peilbuizen_routes(df=df_random_test,
                          start_address=settings['calculation']['startlocation'],
                          route_profile=settings['calculation']['distance_calculation_method'],
                          output_dir=Path(settings['files']['path_results']) / 'test')

Processing peilbuis routes per project.:   0%|          | 0/8 [00:00<?, ?it/s]2025-02-03 11:49:14:INFO:project.utils:Project Holtingerveld _Ootmaanlanden en Koningschut heeft 3 of minder locaties. Optimale route uitrekenen is zinloos.
2025-02-03 11:49:14:INFO:project.utils:Project Mastenbroek-IJssel heeft 3 of minder locaties. Optimale route uitrekenen is zinloos.
2025-02-03 11:49:14:INFO:project.utils:Project Olde Maten en Veerslootlanden heeft 3 of minder locaties. Optimale route uitrekenen is zinloos.
2025-02-03 11:49:14:INFO:project.utils:Project Primair meetnet blok 1 heeft 3 of minder locaties. Optimale route uitrekenen is zinloos.
2025-02-03 11:49:14:INFO:project.utils:Project Primair meetnet blok 2 heeft 3 of minder locaties. Optimale route uitrekenen is zinloos.
2025-02-03 11:49:15:INFO:project.utils:Project Primair meetnet blok 3 heeft 3 of minder locaties. Optimale route uitrekenen is zinloos.
2025-02-03 11:49:15:INFO:project.utils:Project Steenwijk_Kallenkote heeft 3 of min

Project Holtingerveld _Ootmaanlanden en Koningschut heeft 3 of minder locaties. Optimale route uitrekenen is zinloos.
Project Mastenbroek-IJssel heeft 3 of minder locaties. Optimale route uitrekenen is zinloos.
Project Olde Maten en Veerslootlanden heeft 3 of minder locaties. Optimale route uitrekenen is zinloos.
Project Primair meetnet blok 1 heeft 3 of minder locaties. Optimale route uitrekenen is zinloos.
Project Primair meetnet blok 2 heeft 3 of minder locaties. Optimale route uitrekenen is zinloos.
Project Primair meetnet blok 3 heeft 3 of minder locaties. Optimale route uitrekenen is zinloos.
Project Steenwijk_Kallenkote heeft 3 of minder locaties. Optimale route uitrekenen is zinloos.
Project Vecht heeft 3 of minder locaties. Optimale route uitrekenen is zinloos.


In [ ]:
#| export
#| eval: false
# eval set to false, else it will run the code for all projects everytime an nbdev_prepare is run.

process_peilbuizen_routes(df=peilbuizen_df,
                          start_address=settings['calculation']['startlocation'],
                          route_profile=settings['calculation']['distance_calculation_method'],
                          utput_dir=settings['files']['path_results'])

Processing peilbuis routes per project.:   0%|          | 0/45 [00:00<?, ?it/s]

2025-02-03 10:56:11:INFO:project.utils:Successfully processed route for project: ---
Processing peilbuis routes per project.:   2%|▏         | 1/45 [00:00<00:30,  1.45it/s]2025-02-03 10:56:11:INFO:project.utils:Project Bagger Dedemsvaart heeft 3 of minder locaties. Optimale route uitrekenen is zinloos.
2025-02-03 10:56:11:INFO:project.utils:Project Baggeren Beilervaart heeft 3 of minder locaties. Optimale route uitrekenen is zinloos.
2025-02-03 10:56:11:INFO:project.utils:Project Beekmaatregelen Reest heeft 3 of minder locaties. Optimale route uitrekenen is zinloos.


Project Bagger Dedemsvaart heeft 3 of minder locaties. Optimale route uitrekenen is zinloos.
Project Baggeren Beilervaart heeft 3 of minder locaties. Optimale route uitrekenen is zinloos.
Project Beekmaatregelen Reest heeft 3 of minder locaties. Optimale route uitrekenen is zinloos.


2025-02-03 10:56:12:INFO:project.utils:Successfully processed route for project: Boetelerveld_nw
Processing peilbuis routes per project.:  11%|█         | 5/45 [00:01<00:10,  3.77it/s]2025-02-03 10:56:12:INFO:project.utils:Project Brongeb VL-AA heeft 3 of minder locaties. Optimale route uitrekenen is zinloos.


Project Brongeb VL-AA heeft 3 of minder locaties. Optimale route uitrekenen is zinloos.


...

2025-02-03 10:56:29:INFO:project.utils:Successfully processed route for project: Vledder en Leierhooilanden
Processing peilbuis routes per project.:  82%|████████▏ | 37/45 [00:20<00:05,  1.59it/s]2025-02-03 10:56:30:INFO:project.utils:Project ZUIDWOLDE-ZUID heeft 3 of minder locaties. Optimale route uitrekenen is zinloos.
2025-02-03 10:56:30:INFO:project.utils:Project Zandwetering Olst-Zuid_waterberging heeft 3 of minder locaties. Optimale route uitrekenen is zinloos.
2025-02-03 10:56:30:INFO:project.utils:Project dijken Stadsdijken Zwolle heeft 3 of minder locaties. Optimale route uitrekenen is zinloos.
2025-02-03 10:56:30:INFO:project.utils:Project effecten op WHH van gaswinning de Wijk II heeft 3 of minder locaties. Optimale route uitrekenen is zinloos.


Project ZUIDWOLDE-ZUID heeft 3 of minder locaties. Optimale route uitrekenen is zinloos.
Project Zandwetering Olst-Zuid_waterberging heeft 3 of minder locaties. Optimale route uitrekenen is zinloos.
Project dijken Stadsdijken Zwolle heeft 3 of minder locaties. Optimale route uitrekenen is zinloos.
Project effecten op WHH van gaswinning de Wijk II heeft 3 of minder locaties. Optimale route uitrekenen is zinloos.


2025-02-03 10:56:31:INFO:project.utils:Successfully processed route for project: peilbesluit Kostverlorenzijl en Kloosterzijl
Processing peilbuis routes per project.:  93%|█████████▎| 42/45 [00:20<00:01,  2.81it/s]2025-02-03 10:56:31:INFO:project.utils:Project peilbesluit de Kruimels heeft 3 of minder locaties. Optimale route uitrekenen is zinloos.
2025-02-03 10:56:31:INFO:project.utils:Project stedelijk Zwolle heeft 3 of minder locaties. Optimale route uitrekenen is zinloos.


Project peilbesluit de Kruimels heeft 3 of minder locaties. Optimale route uitrekenen is zinloos.
Project stedelijk Zwolle heeft 3 of minder locaties. Optimale route uitrekenen is zinloos.


2025-02-03 10:56:32:INFO:project.utils:Successfully processed route for project: zandwinplassen
Processing peilbuis routes per project.: 100%|██████████| 45/45 [00:21<00:00,  2.07it/s]


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()